In [1]:
# pip install torchvision

In [2]:
# pip install torch

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torchvision.models as models
import matplotlib.pyplot as plt

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

100%|██████████| 26421880/26421880 [00:05<00:00, 4845605.10it/s]


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 29515/29515 [00:00<00:00, 307744.60it/s]


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3395777.22it/s]


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 5148/5148 [00:00<00:00, 430588.22it/s]

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [4]:
labels_map = [
    "T-Shirt",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle Boot",
    ]


In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [6]:
X = torch.rand(1, 28, 28)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([8])


In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.294603  [   64/60000]
loss: 2.281461  [ 6464/60000]
loss: 2.269599  [12864/60000]
loss: 2.263091  [19264/60000]
loss: 2.241760  [25664/60000]
loss: 2.217853  [32064/60000]
loss: 2.213265  [38464/60000]
loss: 2.186796  [44864/60000]
loss: 2.187571  [51264/60000]
loss: 2.147801  [57664/60000]
Test Error: 
 Accuracy: 51.7%, Avg loss: 2.140068 

Epoch 2
-------------------------------
loss: 2.151928  [   64/60000]
loss: 2.135676  [ 6464/60000]
loss: 2.078463  [12864/60000]
loss: 2.090251  [19264/60000]
loss: 2.032611  [25664/60000]
loss: 1.978696  [32064/60000]
loss: 1.995765  [38464/60000]
loss: 1.920676  [44864/60000]
loss: 1.927269  [51264/60000]
loss: 1.845744  [57664/60000]
Test Error: 
 Accuracy: 54.2%, Avg loss: 1.844279 

Epoch 3
-------------------------------
loss: 1.879360  [   64/60000]
loss: 1.846591  [ 6464/60000]
loss: 1.726514  [12864/60000]
loss: 1.768052  [19264/60000]
loss: 1.660658  [25664/60000]
loss: 1.614383  [32064/600

In [10]:
x = test_data[0][0]

In [11]:
y = test_data[0][1]

In [12]:
with torch.no_grad():
    pred=model(x)
    prediction = labels_map[pred[0].argmax(0)]
    truth = labels_map[y]
    print(prediction + "\n" + truth)

Ankle Boot
Ankle Boot


In [13]:
x = test_data[1][0]

In [14]:
y = test_data[1][1]

In [15]:
with torch.no_grad():
    pred=model(x)
    prediction = labels_map[pred[0].argmax(0)]
    truth = labels_map[y]
    print(prediction + "\n" + truth)

Pullover
Pullover


In [16]:
x = test_data[2][0]

In [17]:
y = test_data[2][1]

In [18]:
with torch.no_grad():
    pred=model(x)
    prediction = labels_map[pred[0].argmax(0)]
    truth = labels_map[y]
    print(prediction + "\n" + truth)

Trouser
Trouser


In [19]:
x = test_data[3][0]

In [20]:
y = test_data[3][1]

In [21]:
with torch.no_grad():
    pred=model(x)
    prediction = labels_map[pred[0].argmax(0)]
    truth = labels_map[y]
    print(prediction + "\n" + truth)

Trouser
Trouser
